In [3]:
# %matplotlib inline
import pandas as pd
from fbprophet import Prophet
import numpy as np
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 6
plt.style.use('fivethirtyeight')

In [4]:
raw_data = pd.read_csv("Online_Retail.csv")
raw_data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,01/12/10 08:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,01/12/10 08:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,01/12/10 08:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,01/12/10 08:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,01/12/10 08:26,3.39,17850.0,United Kingdom


In [5]:
sales_data = raw_data.drop(['InvoiceNo','Description','CustomerID','Country','UnitPrice'], axis = 1)
sales_data['InvoiceDate'] = pd.to_datetime(sales_data['InvoiceDate'])
print(sales_data.info())
sales_data.index

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 3 columns):
StockCode      541909 non-null object
Quantity       541909 non-null int64
InvoiceDate    541909 non-null datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 12.4+ MB
None


RangeIndex(start=0, stop=541909, step=1)

In [6]:
stock_codes = list(sales_data.StockCode.unique())

In [7]:
def extract_product(product_id,freq):
    #pull out all transactions of one product and drop StockCode
    product = sales_data.loc[sales_data['StockCode'] == product_id].drop('StockCode',axis=1)
    product=product.set_index(['InvoiceDate']) #Convert to timeseries 

    #DownSampling to a day
    day_summary = pd.DataFrame()
    day_summary['Quantity'] = product.Quantity.resample(freq).sum()
    day_summary = day_summary.fillna(0) #replacing NaN with 0
    day_summary = day_summary.clip(0) #replace -ve values with 0

    day_summary = day_summary['2010-12-12':]#trim as almost no data available before this date
    day_summary['Quantity']+=1#ti avoid -inf while taking log transformation 
#     print(day_summary.describe())
    #plot
#     fig, ax1 = plt.subplots()
#     ax1.plot(day_summary.index, day_summary['Quantity'])
#     ax1.set_xlabel('Date')
#     ax1.set_ylabel('Quantity')
#     # ax1.set_ylim(0,100)
#     plt.show()
    return day_summary

In [101]:
def result_analysis(errors):
    errors = [value[1] for value in errors]
    None_count = errors.count(None)
    errors = [x for x in errors if x!=None]
    min_value = min(errors)
    max_value = max(errors)
    avg_value = sum(errors)/len(errors)
    print("min value = "+str(min_value))
    print("max value = "+str(max_value))
    print("avg value = "+str(avg_value))
    print("none count = "+str(None_count))

In [110]:
def get_rmse_relative_error(y,yhat):
    relative_errors = [(x-y)/x for x,y in zip(y,yhat)]
    root_mean_square_relative_error = np.sqrt(sum([x**2 for x in relative_errors])/len(relative_errors))
    return root_mean_square_relative_error

In [120]:
def get_mean_relative_error(y,yhat):
    relative_error = [abs((x-y)/x) for x,y in zip(y,yhat)]
    mean_relative_error = sum(relative_error)/len(relative_error)
    return mean_relative_error

In [124]:
errors = list()
count =1 
for product in stock_codes:
    print('\r', count, end='')
    count+=1
    freq = 'D'
    try:
        day_summary = extract_product(product,freq)
        # day_summary.head()
        day_summary.reset_index(inplace=True)
        day_summary = day_summary.rename(columns={'Quantity': 'y','InvoiceDate': 'ds'})
        train,test = day_summary[:int(len(day_summary)*0.9)],day_summary[int(len(day_summary)*0.9):]
#         print((len(train),len(test),len(day_summary)))
        my_model = Prophet(daily_seasonality=True,yearly_seasonality=True,weekly_seasonality=True)
        my_model.fit(train)
        future_dates = my_model.make_future_dataframe(periods=len(test),freq=freq)
        forecast = my_model.predict(future_dates)
        predicted = forecast[['ds', 'yhat']].tail(len(test))
#         errors.append((product,get_rmse_relative_error(test.y,predicted.yhat)))
        errors.append((product,get_mean_relative_error(test.y,predicted.yhat)))

    except:
        errors.append((product,None))
#     print(np.sqrt(mean_squared_error(test.y,predicted.yhat)))
#     my_model.plot(forecast,uncertainty=True);
#     my_model.plot_components(forecast);
    
# result_analysis(errors)
with open("result.txt", 'w') as file:
    for item in errors:
        file.write("{}\n".format(item))


 543

INFO:fbprophet.forecaster:n_changepoints greater than number of observations.Using 1.0.


 552

INFO:fbprophet.forecaster:n_changepoints greater than number of observations.Using 15.0.


 978

...

 4033

INFO:fbprophet.forecaster:n_changepoints greater than number of observations.Using 9.0.


 4038

INFO:fbprophet.forecaster:n_changepoints greater than number of observations.Using 6.0.


 4070

In [125]:
errors

[('85123A', 305.98983086560651),
 ('71053', 1.8158211855338162),
 ('84406B', 1.3082517082733069),
 ('84029G', 6.1864034603950788),
 ('84029E', 35.608458176860609),
 ('22752', 1.8357867287315877),
 ('21730', 2.4132165249652093),
 ('22633', 15.338359689862269),
 ('22632', 6.4682133591199156),
 ('84879', 33.583701244902819),
 ('22745', 2.4187231738130279),
 ('22748', 1.4486722249802289),
 ('22749', 2.0643247696170151),
 ('22310', 5.263886104380612),
 ('84969', 0.67054886007828074),
 ('22623', 4.8463696535127854),
 ('22622', 4.1098231242494903),
 ('21754', 1.7702941850117504),
 ('21755', 3.5092438723579416),
 ('21777', None),
 ('48187', 3.222096995951325),
 ('22960', 10.200446498536195),
 ('22913', 2.5991527815447659),
 ('22912', 4.7862626732888032),
 ('22914', 12.045986555203628),
 ('21756', 11.031126509078778),
 ('22728', 23.030000675903391),
 ('22727', 29.278203096330468),
 ('22726', 5.1860308004281075),
 ('21724', 1.0327554168007793),
 ('21883', 8.1892880613958852),
 ('10002', 2.007941

In [126]:
result_analysis(errors)

min value = 0.0
max value = 1872.20679427
avg value = 9.30450912443
none count = 259
